# スクレイピング

## グーグルから

In [23]:
# グーグルから
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
import re
import urllib.request
import os
from tqdm import tqdm
from selenium.webdriver.common.by import By

DRIVER_PATH = 'C:\\Users\\nanan\\Downloads\\chromedriver_win32 (1)\\chromedriver.exe'

options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--proxy-server="direct://"')
options.add_argument('--proxy-bypass-list=*')
options.add_argument('--start-maximized')

def search():
    global driver, actions
    driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)
    actions = ActionChains(driver)
    url = "https://www.google.com/search?q=" + '+'.join(
        query.split()) + "&safe=off&hl=ja&source=lnms&tbm=isch&sa=X"
    driver.get(url)

    while not driver.find_elements(By.CLASS_NAME, "wXeWr"):
        time.sleep(.5)
    driver.find_element(By.CLASS_NAME, "wXeWr").click()


def getLinks():
    global srcs
    more = driver.find_element(By.CLASS_NAME, "mye4qd")
    end = driver.find_element(By.CLASS_NAME, "OuJzKb")
    for i in range(300):
        actions.key_down(Keys.ARROW_RIGHT)
    cnt = 1
    while not (more.is_displayed() or end.is_displayed()):
        if cnt % 5 == 0:
            if len(re.findall("imgurl=(.+?)&amp;", urllib.parse.unquote(driver.page_source))) > max_num + 5:
                break
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        actions.perform()
        time.sleep(1)
        cnt += 1

    if more.is_displayed(): 
        more.click()
    while not end.is_displayed():
        if cnt % 5 == 0:
            if len(re.findall("imgurl=(.+?)&amp;", urllib.parse.unquote(driver.page_source))) > max_num + 5:
                break
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        actions.perform()
        time.sleep(1)
        cnt += 1
    for _ in range(5):
        actions.perform()
        time.sleep(1)
    srcs = re.findall("imgurl=(.+?)&amp;", urllib.parse.unquote(driver.page_source))
    print("Number of links:", len(srcs))  # ログを追加
    driver.close()



def download():
    filename = '_'.join(query.split())
    while True:
        if not os.path.exists(filename):
            os.mkdir(filename)
            break
        else:
            filename += "_"

    for i, src in enumerate(tqdm(srcs[:max_num])):
        ext = src[-4:] if src[-4:] in ('.jpg', '.png', '.gif') else '.png'
        with open(f"{filename}/{filename}{i}{ext}", "wb") as f:
            try:
                f.write(requests.get(src).content)
            except:
                try:
                    with urllib.request.urlopen(src) as u:
                        f.write(u.read())
                except:
                    continue


if __name__ == "__main__":
    query = input("Search:  ")
    max_num = int(input("何枚ダウンロードしますか？（最大）"))
    print("Searching...")
    search()
    print("Done.")
    print("Getting links...")
    getLinks()
    print("Number of links:", len(srcs))  # リンクの数をログに出力
    print("Done.")
    print("Now downloading...")
    download()
    print("Done.")

Search:   小銭
何枚ダウンロードしますか？（最大） 300


Searching...


C:\Users\nanan\AppData\Local\Temp\ipykernel_24900\4033920290.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)


Done.
Getting links...
Number of links: 152
Number of links: 152
Done.
Now downloading...


100%|██████████| 152/152 [01:08<00:00,  2.23it/s]

Done.


## グーグルから（サムネイル）

In [34]:
# サムネイル画像なので画質悪い
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time
import requests
import shutil
import base64

options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--disable-extensions")
options.add_argument('--proxy-server="direct://"')
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
#options.add_argument("--headless")

DRIVER_PATH = "C:\\Users\\nanan\\Downloads\\chromedriver_win32 (1)\\chromedriver.exe" #chromedriverの場所
driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)

query = input('Search word? :')
url = ("https://www.google.com/search?hl=jp&q=" + "+".join(query.split()) + "&btnG=Google+Search&tbs=0&safe=off&tbm=isch")
driver.get(url)

#適当に下までスクロールしてる--
for t in range(8):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1.5)
try:driver.find_element_by_class_name("mye4qd").click() #「検索結果をもっと表示」ってボタンを押してる
except:pass
for t in range(8):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(1.5)

srcs = driver.find_elements(By.XPATH, '//img[@class="rg_i Q4LuWd"]')
try:os.mkdir(query) #検索語と同じ名前のフォルダを作る、保存先
except:pass
#--

i = 0 #ファイル名に通し番号をつける用カウンタ

print("Downloading...")
for j, src in enumerate(srcs):
    if j % 50 == 0 or j == len(srcs)-1:
        print("|"+ ("■" * (20 * j // (len(srcs)-1)))+ (" -" * (20 - 20 * j // (len(srcs)-1)))+ "|",f"{100*j//(len(srcs)-1)}%") #ダウンロードの進捗示すやつ
    file_name = f"{query}/{'_'.join(query.split())}_{str(i).zfill(3)}.jpg" #ファイル名とか場所とか
    src = src.get_attribute("src")
    if src != None:
#画像に変換--
        if "base64," in src:
            with open(file_name, "wb") as f:
                f.write(base64.b64decode(src.split(",")[1]))
        else:
            res = requests.get(src, stream=True)
            with open(file_name, "wb") as f:
                shutil.copyfileobj(res.raw, f)
#--
        i += 1

driver.quit() #ウィンドウを閉じる
print(f"Download is complete. {i} images are downloaded.")

C:\Users\nanan\AppData\Local\Temp\ipykernel_30520\2967242095.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)
C:\Users\nanan\AppData\Local\Temp\ipykernel_30520\2967242095.py:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)


Search word? : 小銭


Downloading...
| - - - - - - - - - - - - - - - - - - - -| 0%
|■ - - - - - - - - - - - - - - - - - - -| 5%
|■■ - - - - - - - - - - - - - - - - - -| 11%
|■■■ - - - - - - - - - - - - - - - - -| 17%
|■■■■ - - - - - - - - - - - - - - - -| 23%
|■■■■■ - - - - - - - - - - - - - - -| 28%
|■■■■■■ - - - - - - - - - - - - - -| 34%
|■■■■■■■■ - - - - - - - - - - - -| 40%
|■■■■■■■■■ - - - - - - - - - - -| 46%
|■■■■■■■■■■ - - - - - - - - - -| 51%
|■■■■■■■■■■■ - - - - - - - - -| 57%
|■■■■■■■■■■■■ - - - - - - - -| 63%
|■■■■■■■■■■■■■ - - - - - - -| 69%
|■■■■■■■■■■■■■■ - - - - - -| 74%
|■■■■■■■■■■■■■■■■ - - - -| 80%
|■■■■■■■■■■■■■■■■■ - - -| 86%
|■■■■■■■■■■■■■■■■■■ - -| 92%
|■■■■■■■■■■■■■■■■■■■ -| 97%
|■■■■■■■■■■■■■■■■■■■■| 100%
Download is complete. 782 images are downloaded.


## bingから

In [38]:
#import文
import glob
import pathlib
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from icrawler.builtin import BingImageCrawler
import cv2
import shutil
import random
from PIL import Image

In [1]:
from icrawler.builtin import BingImageCrawler

coin_list=['一円玉', '五円玉', '十円玉', '五十円玉', '百円玉', '五百円玉', '小銭　画像']

#スクレイピングを行う関数
def scraping(word, max_num, path):
    bing_crawler = BingImageCrawler(downloader_threads = 4, storage = {'root_dir': path})
    bing_crawler.crawl(keyword = word, filters = None, offset = 0, max_num = max_num,
                       min_size=(200, 200), max_size = None)
# スクレイピング実行
for i in range(len(coin_list)):
    scraping(coin_list[i], 5, './con/' + coin_list[i] + '/')

2023-05-29 17:48:58,477 - INFO - icrawler.crawler - start crawling...
2023-05-29 17:48:58,479 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-05-29 17:48:58,480 - INFO - feeder - thread feeder-001 exit
2023-05-29 17:48:58,483 - INFO - icrawler.crawler - starting 1 parser threads...
2023-05-29 17:48:58,484 - INFO - icrawler.crawler - starting 4 downloader threads...
2023-05-29 17:48:59,136 - INFO - parser - parsing result page https://www.bing.com/images/async?q=一円玉&first=0
2023-05-29 17:48:59,158 - INFO - downloader - skip downloading file 000001.jpg
2023-05-29 17:48:59,162 - INFO - downloader - skip downloading file 000002.jpg
2023-05-29 17:48:59,162 - INFO - downloader - skip downloading file 000003.jpg
2023-05-29 17:48:59,164 - INFO - downloader - skip downloading file 000004.jpg
2023-05-29 17:48:59,165 - INFO - downloader - skip downloading file 000005.jpg
2023-05-29 17:48:59,913 - INFO - downloader - downloaded images reach max num, thread downloader-002 is ready to 

In [40]:
# スクレイピング実行
for i in range(len(coin_list)):
    scraping(coin_list[i], 500, './coin/' + coin_list[i] + '/')

2023-05-23 14:12:10,506 - INFO - icrawler.crawler - start crawling...
2023-05-23 14:12:10,507 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-05-23 14:12:10,508 - INFO - icrawler.crawler - starting 1 parser threads...
2023-05-23 14:12:10,509 - INFO - icrawler.crawler - starting 4 downloader threads...
2023-05-23 14:12:11,346 - INFO - parser - parsing result page https://www.bing.com/images/async?q=一円玉&first=0
2023-05-23 14:12:11,513 - INFO - downloader - image #1	https://livedoor.blogimg.jp/mura_bito/imgs/5/5/550eb111.jpg
2023-05-23 14:12:11,514 - INFO - downloader - image #2	https://antique-coin.info/kaitori/wp-content/uploads/2018/03/2-3.jpg
2023-05-23 14:12:11,539 - INFO - downloader - image #3	https://antique-coin.info/kaitori/wp-content/uploads/2018/03/1-4.jpg
2023-05-23 14:12:11,679 - INFO - downloader - image #4	https://blogimg.goo.ne.jp/user_image/74/ad/f2408d535fa9dce993610b75b33e5a72.jpg
2023-05-23 14:12:11,693 - INFO - downloader - image #5	https://blogimg.goo.

# ファイル整理

In [57]:
import glob
import os

files = glob.glob("C:\\Users\\nanan\\OneDrive\\ドキュメント\\GitHub\\memo\\coin\\小銭\\*.png")
for i, old_name in enumerate(files):
    # 新しいファイル名
    new_name = "each_{0:03d}.jpg".format(i + 1)
    # ファイル名の変更
    os.rename(old_name, new_name)

C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\一円玉15.png → each_091.jpg
C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\十円玉10.png → each_092.jpg
C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\十円玉171.png → each_093.jpg
C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\小銭3.png → each_094.jpg
C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\小銭4.png → each_095.jpg
C:\Users\nanan\OneDrive\ドキュメント\GitHub\memo\coin\小銭\小銭8.png → each_096.jpg


In [9]:
import shutil
import glob
#別フォルダへ移動
in_dir = "C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\coin_data_all_label\\test\\images\\*.txt"
in_img = glob.glob(in_dir)
for t in range(len(in_img) // 1):
    shutil.move(in_img[t], "C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\coin_data_all_label\\test\\labels")

# データ拡張

In [1]:
import os
import random
import cv2
import albumentations as A
from matplotlib import pyplot as plt
import copy
import numpy as np

In [138]:
class Data:

    rotate90_transform = A.Compose([
     A.Affine(rotate=[90, 90], p=1, mode=cv2.BORDER_CONSTANT, fit_output=True)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    rotate180_transform = A.Compose([
     A.Affine(rotate=[180, 180], p=1, mode=cv2.BORDER_CONSTANT, fit_output=True)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    rotate270_transform = A.Compose([
     A.Affine(rotate=[270, 270], p=1, mode=cv2.BORDER_CONSTANT, fit_output=True)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    blur_transform = A.Compose([
      A.Blur(blur_limit=31, p=1),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    dark_transform = A.Compose([
      A.RandomBrightness(limit=[-0.2,-0.2], p=1),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    def __init__(self, image="", bboxes=0, id="", class_labels=[]):
        self.image = image
        self.bboxes = bboxes
        self.id = id
        self.class_labels = class_labels

# jpgとtxtからデータをインポートするメソッド
    def importdata(self, imgpath):
        dirpath = os.path.dirname(imgpath)[:-7]
        id = os.path.splitext(os.path.basename(imgpath))[0]
        txtpath = dirpath + f"/labels/{id}.txt"

        img = cv2.imread(imgpath)
        self.image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        bboxes = []
        class_labels = []  # クラスラベルを格納するリストを作成

        with open(txtpath) as f:
            for line in f:
                line_list = line.split(" ")
                class_label = int(line_list[0])  # txtファイルの先頭がクラスラベル
                bbox = line_list[1:]
                bbox = [float(i.replace('\n', '')) for i in bbox]
                bboxes.append(bbox)
                class_labels.append(class_label)  # クラスラベルをリストに追加

        self.bboxes = bboxes
        self.label = 0
        self.id = id
        self.class_labels = class_labels


# albumentationsで変換したデータをインポートするメソッド
    def import_transformdata(self, transform_data, origin_data, process):
        self.image = transform_data["image"]
        self.bboxes = transform_data["bboxes"]
        self.label = 0
        self.id = origin_data.id + "_" + process
        self.class_labels = transform_data["class_labels"]
  
# 90度回転メソッド
    def rotate90(self):
        rotate90_transformed = Data.rotate90_transform(image=self.image, 
                                                        bboxes=self.bboxes, 
                                                        class_labels=self.class_labels)
        image =rotate90_transformed["image"]
        bboxes = rotate90_transformed["bboxes"]
        id = self.id + "_rot90"
        class_labels = rotate90_transformed["class_labels"]
        rot90data = Data(image, bboxes, id, class_labels)
        return rot90data


# 反時計回りに180ﾟ回転したデータを返すメソッド
    def rotate180(self):
        rotate180_transformed = Data.rotate180_transform(image=self.image, 
                                                        bboxes=self.bboxes, 
                                                        class_labels=self.class_labels)
        image =rotate180_transformed["image"]
        bboxes = rotate180_transformed["bboxes"]
        id = self.id + "_rot180"
        class_labels = rotate180_transformed["class_labels"]
        rot180data = Data(image, bboxes, id, class_labels)
        return rot180data

# 反時計回りに270ﾟ回転したデータを返すメソッド
    def rotate270(self):
        rotate270_transformed = Data.rotate270_transform(image=self.image, 
                                                         bboxes=self.bboxes, 
                                                         class_labels=self.class_labels)
        image =rotate270_transformed["image"]
        bboxes = rotate270_transformed["bboxes"]
        id = self.id + "_rot270"
        class_labels = rotate270_transformed["class_labels"]
        rot270data = Data(image, bboxes, id, class_labels)
        return rot270data

# blur
    def blur_method(self):
        blur_transformed = Data.blur_transform(image=self.image, 
                                                bboxes=self.bboxes, 
                                                class_labels=self.class_labels)
        image =blur_transformed["image"]
        bboxes = blur_transformed["bboxes"]
        id = self.id + "_blur"
        class_labels = blur_transformed["class_labels"]
        blurdata = Data(image, bboxes, id, class_labels)
        return blurdata

# dark
    def dark_method(self):
        dark_transformed = Data.dark_transform(image=self.image, 
                                                bboxes=self.bboxes, 
                                                class_labels=self.class_labels)
        image =dark_transformed["image"]
        bboxes = dark_transformed["bboxes"]
        id = self.id + "_dark"
        class_labels = dark_transformed["class_labels"]
        darkdata = Data(image, bboxes, id, class_labels)
        return darkdata

# 指定のパスにjpgとtxtファイルでデータ保存するメソッド
    def export_data(self, imgdirpath):
        id = self.id
        dirpath = imgdirpath[:-7]
        export_imgpath = imgdirpath + f"/{id}.jpg"
        export_txtpath = dirpath + f"/labels/{id}.txt"

        img = cv2.cvtColor(self.image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(export_imgpath, img)
        
        txt = ""
        for class_label, bbox in zip(self.class_labels, self.bboxes):
            x_min, y_min, width, height = [i for i in bbox]
            line = f"{class_label} {x_min} {y_min} {width} {height}"
            txt += line + "\n"
            print(line)

        f = open(export_txtpath, 'w')
        f.write(txt) 
        f.close()

# 画像とバウンディングボックスを表示するメソッド
    def visualize(self, img_width, img_height, figsize = (10,10)):

        for bbox in self.bboxes:
            x_mid_nor, y_mid_nor, width_nor, height_nor  = [float(i) for i in bbox]

            width = width_nor * img_width  
            height = height_nor * img_height   

            x_min = x_mid_nor * img_width - width/2   
            y_min = y_mid_nor * img_height - height/2    
            x_max = x_min + width
            y_max = y_min + height

            x_min = int(x_min)
            x_max = int(x_max)
            y_min = int(y_min)
            y_max = int(y_max)

            img = cv2.rectangle(self.image,
                              pt1=(x_min, y_min),
                              pt2=(x_max, y_max),
                              color=(255, 0, 0),
                              thickness=3)

        plt.figure(figsize = figsize)
        plt.axis('off')
        plt.imshow(img)

# データ定義

In [141]:
dataset_origin = []
imgdirpath = ".\\coin_data_all_label\\train\\images"
for filename in os.listdir(imgdirpath):
  fullpath = imgdirpath + "\\" + filename
  data = Data()
  data.importdata(fullpath)
  dataset_origin.append(data)

# データの可視化
#dataset_origin[0].visualize(1600, 1066)

In [142]:
dataset = copy.deepcopy(dataset_origin)

In [143]:
rotate90_dataset = [data.rotate90() for data in dataset]    #反時計周りに90ﾟ回転
rotate180_dataset = [data.rotate180() for data in dataset] #反時計周りに180ﾟ回転
rotate270_dataset = [data.rotate270() for data in dataset]  #反時計周りに270ﾟ回転
blur_dataset = [data.blur_method() for data in dataset] # ぼかし
dark_dataset = [data.dark_method() for data in dataset] # 暗くする
# データセットに上記処理したデータを追加
dataset.extend(rotate90_dataset)
dataset.extend(rotate180_dataset)
dataset.extend(rotate270_dataset)
dataset.extend(blur_dataset)
dataset.extend(dark_dataset)

In [144]:
for data in dataset:
  data.export_data(".\\dataset_alllabel\\check\\images")

0 0.687813 0.321295 0.148125 0.23546
0 0.786875 0.587711 0.155 0.237336
0 0.6725 0.765009 0.1575 0.229831
1 0.61 0.536585 0.17 0.243902
2 0.47 0.751407 0.185 0.273921
4 0.475625 0.381332 0.17375 0.269231
4 0.343438 0.556285 0.166875 0.262664
4 0.745313 0.554348 0.16875 0.205314
1 0.752344 0.291063 0.148438 0.161836
0 0.613281 0.351449 0.132812 0.152174
2 0.399219 0.728261 0.157812 0.210145
2 0.288281 0.577295 0.154688 0.198068
0 0.660156 0.161836 0.132812 0.140097
0 0.836719 0.376812 0.145313 0.169082
2 0.235937 0.735507 0.15625 0.219807
2 0.189844 0.600242 0.132812 0.190821
0 0.40625 0.349903 0.185 0.290448
0 0.231875 0.701754 0.17875 0.2846
1 0.41625 0.661793 0.2025 0.317739
4 0.2075 0.384016 0.2025 0.335283
0 0.45125 0.477583 0.1875 0.25731
0 0.597074 0.48425 0.110372 0.1645
0 0.832447 0.6695 0.109043 0.165
4 0.837267 0.45 0.132646 0.19
4 0.699136 0.684 0.124335 0.194
2 0.659076 0.27325 0.129987 0.1955
1 0.535406 0.33475 0.122673 0.1815
0 0.710106 0.477 0.108378 0.157
2 0.214062 0.5

# 諸々

In [1]:
import wandb

In [1]:
!wandb login

wandb: Currently logged in as: hayato-miura. Use `wandb login --relogin` to force relogin


In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

# 学習の実行

In [6]:
!python ./train.py --workers 8 --device 0 --batch-size 4 --epochs 100 --data ./data/mydata.yaml --cfg ./cfg/training/yolov7.yaml --weight ./yolov7.pt --name ./dataset_alllabel --hyp ./data/hyp.scratch.p5.yaml


autoanchor: Analyzing anchors... anchors/target = 4.90, Best Possible Recall (BPR) = 1.0000
                 all          91         225       0.201       0.553       0.216      0.0934
                 all          91         225       0.347        0.44       0.275       0.128
                 all          91         225       0.311       0.787       0.567        0.43
                 all          91         225       0.521       0.788       0.629       0.539
                 all          91         225       0.859        0.74       0.841        0.71
                 all          91         225       0.914       0.831       0.923       0.775
                 all          91         225       0.837       0.887       0.917        0.79
                 all          91         225       0.844       0.872       0.903       0.795
                 all          91         225       0.813       0.922       0.928       0.812
                 all          91         225       0.888       0.894  

YOLOR  v0.1-122-g3b41c2c torch 2.0.0 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8191.5625MB)

Namespace(weights='./yolov7.pt', cfg='./cfg/training/yolov7.yaml', data='./data/mydata.yaml', hyp='./data/hyp.scratch.p5.yaml', epochs=100, batch_size=4, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='./dataset_alllabel', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\dataset_alllabel2', total_batch_size=4)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005,

In [8]:
from collections import defaultdict
import os
from glob import glob

LABEL_PATH="C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\yolov7\\dataset_alllabe\\labels\\train\\"
# 画像とアノテーション、それぞれのデータを取得
lblAry = sorted([lbl_path for lbl_path in glob(os.path.join(LABEL_PATH,'*.txt'))])

# アノテーションしたクラスラベルの数を数える
dict = defaultdict(int)
for lbl_path in glob(os.path.join(LABEL_PATH,'*.txt')):
    if 'class' in lbl_path:
        continue
    with open(lbl_path, 'r') as rf:
        lines = rf.readlines()
        for line in lines:
            line = line.rstrip('\n')
            num = int(line.split()[0])
            dict[num] += 1
print(dict)

defaultdict(<class 'int'>, {0: 822, 1: 852, 2: 918, 4: 840, 3: 588, 5: 900})


In [ ]:
# 画像を読み込んでモデルを適用
import os
from glob import glob

"""定数定義"""
DATA_PATH = os.path.join('..','data')
TEST_IMAGES_PATH = os.path.join(DATA_PATH,'test_images')
OUTPUT_FOLDER_PATH = os.path.join(DATA_PATH,'detect')
"""実行"""
img_pathAry = sorted([img_path for img_path in glob(os.path.join(TEST_IMAGES_PATH,'*'))])

In [4]:
%pwd

'C:\\Users\\nanan\\anaconda3\\yolov7'

# 推論

In [7]:
!python detect_for_webcamera.py --weights runs/train/dataset_alllabel2/weights/best.pt --source dataset_alllabel/images/test --name test_result

Namespace(weights=['runs/train/dataset_alllabel2/weights/best.pt'], source='dataset_alllabel/images/test', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='test_result', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

4 ones, 1 five, 5 tens, 1 fifty, 7 one_hundreds, 2 five_hundreds, Done. (30.0ms) Inference, (7.0ms) NMS
 The image with the result is saved in: runs\detect\test_result3\1.jpg
5 ones, 4 fives, 7 tens, 4 fiftys, 8 one_hundreds, 5 five_hundreds, Done. (28.0ms) Inference, (4.0ms) NMS
 The image with the result is saved in: runs\detect\test_result3\2.jpg
2 ones, 1 five, 2 tens, 1 fifty, Done. (29.0ms) Inference, (4.0ms) NMS
 The image with t

YOLOR  v0.1-122-g3b41c2c torch 2.0.0 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8191.5625MB)

Model Summary: 314 layers, 36508742 parameters, 6194944 gradients
C:\Users\nanan\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# yolov8

In [1]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")
results = model("https://ultralytics.com/images/bus.jpg",save=True) 

100%|██████████| 49.7M/49.7M [00:01<00:00, 26.7MB/s]

100%|██████████| 476k/476k [00:00<00:00, 44.3MB/s]
image 1/1 C:\Users\nanan\anaconda3\yolov7\bus.jpg: 640x480 4 persons, 1 bus, 57.5ms
Speed: 2.0ms preprocess, 57.5ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")
results = model.train(data="C:\\Users\\nanan\\anaconda3\\ultralytics\\ultralytics\\yolo\\data\\datasets\\coin\\data.yaml", epochs=100 ,batch=8) 

New https://pypi.org/project/ultralytics/8.0.111 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.109  Python-3.10.9 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\Users\nanan\anaconda3\ultralytics\ultralytics\yolo\data\datasets\coin\data.yaml, epochs=100, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=F

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks failed . Anomalies were detected with AMP on your system that may lead to NaN losses or zero-mAP results, so AMP will be disabled during training.
optimizer: SGD(lr=0.01) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias
train: Scanning C:\Users\nanan\anaconda3\ultralytics\ultralytics\yolo\data\datasets\coin\train\labels.cache... 1794 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1794/1794 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning C:\Users\nanan\anaconda3\ultralytics\ultralytics\yolo\data\datasets\coin\valid\labels.cache... 91 images, 0 backgrounds, 0 corrupt: 100%|██████████| 91/91 [00:00<?, ?it/s]
Plotting labels to runs\detect\train\labels.jpg... 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to r

In [22]:
from models.experimental import attempt_load
import argparse
from detect import detect 
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer

In [23]:
weights="C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\yolov7\\runs\\train\\dataset_alllabel2\\weights\\best.pt"
device="cuda"
#weights=opt.weights
model = attempt_load(weights, map_location=device)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [20]:
from ultralytics import YOLO
#model = YOLO("C:\\Users\nanan\\anaconda3\\yolov7\\runs\\detect\\train\\weights\\best.pt") 
model = YOLO(".\\runs\\detect\\train\\weights\\best.pt") 

In [27]:
model("C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\ultralytics\\ultralytics\\yolo\\data\\datasets\\coin\\test\\images",save=True, conf=0.5, iou=0.4)


image 1/25 C:\Users\nanan\OneDrive\document\GitHub\memo\ultralytics\ultralytics\yolo\data\datasets\coin\test\images\1.jpg: 448x640 3 ones, 1 five, 5 tens, 1 fifty, 7 one_hundreds, 2 five_hundreds, 68.5ms
image 2/25 C:\Users\nanan\OneDrive\document\GitHub\memo\ultralytics\ultralytics\yolo\data\datasets\coin\test\images\2.jpg: 480x640 5 ones, 4 fives, 7 tens, 3 fiftys, 8 one_hundreds, 5 five_hundreds, 69.0ms
image 3/25 C:\Users\nanan\OneDrive\document\GitHub\memo\ultralytics\ultralytics\yolo\data\datasets\coin\test\images\3.jpg: 512x640 2 ones, 1 five, 1 ten, 1 fifty, 1 five_hundred, 73.5ms
image 4/25 C:\Users\nanan\OneDrive\document\GitHub\memo\ultralytics\ultralytics\yolo\data\datasets\coin\test\images\each_045.jpg: 480x640 2 ones, 2 fives, 2 tens, 2 fiftys, 2 one_hundreds, 4 five_hundreds, 49.5ms
image 5/25 C:\Users\nanan\OneDrive\document\GitHub\memo\ultralytics\ultralytics\yolo\data\datasets\coin\test\images\fifty_006.jpg: 640x640 1 fifty, 62.2ms
image 6/25 C:\Users\nanan\OneDrive\

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'one', 1: 'five', 2: 'ten', 3: 'fifty', 4: 'one_hundred', 5: 'five_hundred'}
 orig_img: array([[[ 89,  78,  78],
         [ 86,  75,  74],
         [ 81,  70,  69],
         ...,
         [ 27,  49, 100],
         [ 27,  48, 102],
         [ 27,  47, 103]],
 
        [[ 51,  40,  39],
         [ 53,  42,  41],
         [ 58,  47,  46],
         ...,
         [ 20,  42,  93],
         [ 20,  41,  95],
         [ 20,  40,  96]],
 
        [[ 99,  87,  87],
         [102,  91,  90],
         [107,  96,  95],
         ...,
         [ 18,  40,  91],
         [ 18,  39,  92],
         [ 18,  38,  94]],
 
        ...,
 
        [[  8,   4,   2],
         [  9,   5,   3],
         [ 10,   6,   4],
         ...,
         [ 11,  40,  81],
         [  0,  21,  62],
         [ 19,  48,  89]],
 
        [[ 27,  23,  22],
 

In [22]:
from algorithm.object_detector import YOLOv7
from utils.detections import draw
import json
import cv2


webcam = cv2.VideoCapture(0)

if webcam.isOpened() == False:
	print('[!] error opening the webcam')

try:
    while webcam.isOpened():
        ret, frame = webcam.read()
        if ret == True:
            detections = yolov7.detect(frame)
            detected_frame = draw(frame, detections)
            print(json.dumps(detections, indent=4))
            cv2.imshow('webcam', detected_frame)
            cv2.waitKey(1)
        else:
            break
except KeyboardInterrupt:
    pass

webcam.release()
print('[+] webcam closed')
yolov7.unload()

ModuleNotFoundError: No module named 'algorithm'

In [24]:
import cv2
import torch
#from models.yolo import Model
from utils import *

# Yoloモデルの初期化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = Model().to(device)
#model.load_state_dict(torch.load("C:\\Users\\nanan\\OneDrive\\document\\GitHub\\memo\\yolov7\\runs\\train\\dataset_alllabel2\\weights\\best.pt"))  # 重みファイルのパスに適宜変更する

# 小銭のクラスラベルと金額の対応表
coin_labels = {
    0: {"label": "1円玉", "amount": 1},
    1: {"label": "5円玉", "amount": 5},
    2: {"label": "10円玉", "amount": 10},
    3: {"label": "50円玉", "amount": 50},
    4: {"label": "100円玉", "amount": 100},
    5: {"label": "500円玉", "amount": 500},
    # 他の小銭のラベルと金額を追加する
}

# カメラのキャプチャ
cap = cv2.VideoCapture(0)  # カメラのインデックスに適宜変更する

while True:
    ret, frame = cap.read()  # フレームのキャプチャ

    # Yoloモデルを使用して小銭を検出
    img = detect(frame)
    img = img.to(device)
    detections = model(img)
    detections = non_max_suppression(detections)

    # 検出結果の処理
    total_amount = 0
    for detection in detections:
        if detection is not None:
            for x_min, y_min, x_max, y_max, class_id, confidence in detection:
                label = coin_labels[int(class_id)]
                amount = label["amount"]
                total_amount += amount

                # 検出結果の描画
                label_text = f"{label['label']}: {amount}円"
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, label_text, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 合計金額の表示
    total_text = f"合計金額: {total_amount}円"
    cv2.putText(frame, total_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # フレームの表示
    cv2.imshow("Coin Detection", frame)

    # 終了条件
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 後処理
cap.release()
cv2.destroyAllWindows()


NameError: name 'detect' is not defined